In [1]:
from PIL import Image
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel, SegformerImageProcessor, AutoModelForSemanticSegmentation , AutoFeatureExtractor
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import weaviate
import numpy as np
from numpy.linalg import norm
import json
import cv2
import base64
from time import sleep

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
checkpoint = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(checkpoint)
processor = CLIPProcessor.from_pretrained(checkpoint)
seg_processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
seg_model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")

def getTextEmbeddings(text):
	inputs = processor(text=text , images=Image.new('RGB' , (72 , 72)), return_tensors="pt", padding=True)
	outputs = model(**inputs , return_dict=True)
	return outputs["text_embeds"]

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [9]:

embeddings = getTextEmbeddings("brown crop top with cargo pants")

In [ ]:
def cosine_similarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    return dot_product / (norm1 * norm2)

In [4]:
client = weaviate.Client(url="http://localhost:8080")

In [5]:
where_filter1 = {
    "path" : ["category"],
    "operator" : "Equal",
    "valueText" : "girls_collegewear"
}

In [10]:
response1 = (
    client.query
    # .get("FlipkartProducts",["uRL", "brand", "category", "product", "price", "rating", "numberRatings", "colour", "row"])
    .get("PinterestImages", ["image", "top{... on PinterestTop { image, _additional {vector} }}"])
    .with_near_vector({"vector" : embeddings.tolist()[0]})
    .with_where(where_filter1)
    # .with_near_vector({"vector" : top_embedding})
    .with_additional(["vector", "id", "distance"])
    .with_limit(7)
    .do()
)

In [13]:
top_embedding = response1["data"]["Get"]["PinterestImages"][0]["top"][0]['_additional']["vector"]

In [14]:
where_filter = {
    "path": ["category"],
    "operator": "Equal",
    "valueText" : "Girls Tops"

}

In [1]:
response = (
    client.query
    .get("FlipkartProducts",["uRL", "brand", "category", "product", "price", "rating", "numberRatings", "colour", "row", "image"])
    .with_where(where_filter)
    .with_near_vector({"vector" : top_embedding})
    # .with_near_vector({"vector" : embeddings.tolist()[0]})
    .with_additional(["vector", "id", "distance"])
    .with_limit(5)
    .do()
)

NameError: name 'client' is not defined

In [12]:
for i in range(15):
    try:
        # image = response["data"]["Get"]["FlipkartProducts"][i]['image']
        image = response1["data"]["Get"]["PinterestImages"][i]["top"][0]['image']
        # image = response1["data"]["Get"]["PinterestImages"][i]['image']
        image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
        image.show()
    except:
        pass

In [ ]:
for i in range(15):
    try:
        image = response["data"]["Get"]["FlipkartProducts"][i]['image']
        # image = response1["data"]["Get"]["PinterestImages"][i]["top"][0]['image']
        # image = response1["data"]["Get"]["PinterestImages"][i]['image']
        image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
        image.show()
    except:
        pass